In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [91]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from ethnicolr import census_ln, pred_census_ln

import warnings
warnings.filterwarnings('ignore')

print('Libraries imported.')

Libraries imported.


In [ ]:
# Where should I bring my friend?

In [6]:
# Init login information 

In [421]:
CLIENT_ID = '5PFGDSXXUVZQ4I32OPZOYZDR0AU3YX5X2DY14TGHFRW0YHHW' # your Foursquare ID
CLIENT_SECRET = 'RYKT0PNIQZGKUB5CWQXNPXAILAUOPS3KKSNVQIV5HBKCORGY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5PFGDSXXUVZQ4I32OPZOYZDR0AU3YX5X2DY14TGHFRW0YHHW
CLIENT_SECRET:RYKT0PNIQZGKUB5CWQXNPXAILAUOPS3KKSNVQIV5HBKCORGY


In [422]:
# Step one, find target location of interest

In [442]:
latitude = 38.539402
longitude = -121.750896
search_query = 'Restaurant'
radius = 6000


print(latitude, longitude)

38.539402 -121.750896


In [443]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [444]:
results = requests.get(url).json()

In [445]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [446]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


In [450]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


In [448]:
dataframe_filtered = dataframe_filtered.fillna('Unknown')
dataframe_filtered = dataframe_filtered[dataframe_filtered['categories'].str.contains('Chinese ')]

In [451]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Davis Sushi Buffet Japanese Restaurant,Sushi Restaurant,707 2nd St,US,Davis,United States,btw F & G,1089,"[707 2nd St (btw F & G), Davis, CA 95616, Unit...","[{'label': 'display', 'lat': 38.54362216518046...",38.543622,-121.739603,NaN,95616,CA,4b3d1938f964a520e78d25e3
1,Symposium Greek Restaurant and Pizza,Greek Restaurant,1620 E 8th St,US,Davis,United States,NaN,2265,"[1620 E 8th St, Davis, CA 95616, United States]","[{'label': 'display', 'lat': 38.55274914580334...",38.552749,-121.731248,NaN,95616,CA,4b36d04df964a520ea3c25e3
2,KetMoRee Thai Restaurant & Bar,Thai Restaurant,238 G St,US,Davis,United States,at 3rd St,1192,"[238 G St (at 3rd St), Davis, CA 95616, United...","[{'label': 'display', 'lat': 38.54476909088795...",38.544769,-121.739045,NaN,95616,CA,4a8770ebf964a5205e0420e3
3,Hunan Bar & Restaurant,Chinese Restaurant,207 D St,US,Davis,United States,at 2nd St,856,"[207 D St (at 2nd St), Davis, CA 95616, United...","[{'label': 'display', 'lat': 38.54337769930769...",38.543378,-121.742473,NaN,95616,CA,4b7f3abff964a520d22030e3
4,Seasons Restaurant,American Restaurant,102 F St,US,Davis,United States,at 1st St,1049,"[102 F St (at 1st St), Davis, CA 95616, United...","[{'label': 'display', 'lat': 38.54241163874384...",38.542412,-121.739477,NaN,95616,CA,4b848e2af964a520873b31e3
5,Yeti Restaurant,Indian Restaurant,234 E St,US,Davis,United States,2nd St,1015,"[234 E St (2nd St), Davis, CA 95616, United St...","[{'label': 'display', 'lat': 38.54421814743618...",38.544218,-121.740994,NaN,95616,CA,4fb700d5e4b0e951f09b72a0
6,Four Seasons Gourmet Chinese Restaurant,Chinese Restaurant,1601 Research Park Dr,US,Davis,United States,NaN,1642,"[1601 Research Park Dr, Davis, CA 95618, Unite...","[{'label': 'display', 'lat': 38.54030800684315...",38.540308,-121.732067,NaN,95618,CA,51f5cd77e4b07911087f4e62
7,Cindy's Restaurant/Kow Thai Restaurant,Breakfast Spot,4823 Chiles Rd,US,Davis,United States,NaN,5159,"[4823 Chiles Rd, Davis, CA 95618, United States]","[{'label': 'display', 'lat': 38.55113560608923...",38.551136,-121.693570,NaN,95618,CA,4b0586b8f964a5202f6b22e3
8,Hibachi Japanese Fast Food Restaurant,Food,403 3rd St,US,Davis,United States,NaN,915,"[403 3rd St, Davis, CA 95616, United States]","[{'label': 'display', 'lat': 38.544402, 'lng':...",38.544402,-121.742542,NaN,95616,CA,4f32778119836c91c7da9b9c
9,Soga's Restaurant,Food,217 E St,US,Davis,United States,NaN,985,"[217 E St, Davis, CA 95616, United States]","[{'label': 'display', 'lat': 38.544295, 'lng':...",38.544295,-121.741457,NaN,95616,CA,4f32962419836c91c7e6472f


In [455]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=5) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=5,
    color='red',
    popup='Target',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [558]:
venue = []
for venue_id in dataframe_filtered.id:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    venue.append(requests.get(url))

In [516]:
venue_id = dataframe_filtered.id[1]

In [521]:
LIMIT = 100
VERSION = '20181204'
url_t = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION,LIMIT)
temp = requests.get(url_t)



In [559]:
df_venue = {}
for venue_file in venue:
    venue_name = json.loads(venue_file.text)
    user_name = []
    print(venue_name['response']['venue']['name'])
    for usrs in venue_name['response']['venue']['tips']['groups']:
        for usr in usrs['items']:
            try:
                print('comment-'+usr['user']['firstName'] + ' ' +usr['user']['lastName'])
                user_name.append(usr['user']['lastName'])
            except KeyError:
                print('comment- anonymous')
            
    for like_usrs in venue_name['response']['venue']['likes']['groups']:
        for like_usr in like_usrs['items']:
            try:
                print('like- '+like_usr['firstName']+ ' ' +like_usr['lastName'])
                user_name.append(like_usr['user']['firstName'])
            except KeyError:
                print('like- anonymous')
            
    for photo_usrs in venue_name['response']['venue']['photos']['groups']:
        for photo_usr in photo_usrs['items']:
            try:
                print('photo- '+photo_usr['user']['firstName'] +' ' +photo_usr['user']['lastName'] )
                user_name.append(photo_usr['user']['firstName'])
            except KeyError:
                print('photo- anonymous')
            
    for v_usrs in venue_name['response']['venue']['listed']['groups']:
        for v_usr in v_usrs['items']:
            try:
                print('visited- '+v_usr['user']['firstName'] +' ' + v_usr['user']['lastName'])
                user_name.append(v_usr['user']['firstName'])
            except KeyError:
                print('visited- anonymous')

    df_venue[venue_name['response']['venue']['name']] = user_name
    
    
    try:
        print('Rating: ' + str(venue_name['response']['venue']['rating']))
    except KeyError:
        print('No rating yet')
    try:
        print('Likes: ' + str(venue_name['response']['venue']['likes']['count']))
    except KeyError:
        print('No likes yet')
    print('\n')

Davis Sushi Buffet Japanese Restaurant
comment-Bryan Reamer
comment-Rex Wui
photo- Tam Do
photo- Marissa Pimentel
visited- Ellyse Look
visited- Tony Shayne
visited- Andrew Hoang
visited- Loida M
Rating: 6.5
Likes: 15


Symposium Greek Restaurant and Pizza
comment-Meredith Holmes
comment- anonymous
photo- Daniel Orton
photo- Kayla Fox
visited- Teresa C
visited- anonymous
Rating: 8.3
Likes: 10


KetMoRee Thai Restaurant & Bar
comment-Garrick C
comment-Jason Unck
photo- Aziza McWhirter
photo- anonymous
visited- anonymous
visited- Kris Gb
Rating: 7.3
Likes: 30


Hunan Bar & Restaurant
comment-Brian Leung
comment-Bev Sykes
photo- Ilian Georgiev
photo- Ilian Georgiev
visited- Rebecca DePalma
visited- anonymous
Rating: 7.4
Likes: 10


Seasons Restaurant
comment-Ardalan Zandian
comment-Teree Castanias
photo- anonymous
photo- Jess H
visited- Ellyse Look
visited- Ellyse Look
Rating: 7.3
Likes: 13


Yeti Restaurant
comment-Alexei Grinbaum
comment- anonymous
photo- Christine Lee
photo- Roger Moy
v

In [560]:
for venue in df_venue.keys():
    df_name = pred_census_ln(pd.DataFrame({'name':df_venue[venue]}),'name')
    

In [561]:
df_name

,name,race,api,black,hispanic,white
0,Whitmer,white,0.002298,0.088970,0.015957,0.892776
1,Smith,white,0.002019,0.247235,0.014485,0.736260
2,Lenny,white,0.010181,0.025251,0.011215,0.953353
3,emuchico,white,0.019694,0.007625,0.008066,0.964615
4,Ellyse,white,0.005008,0.050107,0.018264,0.926621
5,Jess,white,0.006230,0.111001,0.023227,0.859542
6,Karen,white,0.035255,0.011924,0.013332,0.939489
